# Task 4 - Link Prediction Baselines

before throwing neural nets at the problem, I try to understand what we're predicting
and see how far simple methods go. every ML result needs a baseline to mean anything.

link prediction: given a knowledge graph with some edges missing, can we predict them.
for each test triplet (h, r, ?), rank all possible tail entities. lower rank is a better prediction.

In [ ]:
import numpy as np
from collections import defaultdict, Counter
import sys, os

sys.path.insert(0, os.path.dirname(os.path.abspath('.')))
from task4_utils import load_triplets, build_mappings, triplets_to_ids, compute_metrics

## load and explore the data

In [ ]:
TRAIN_PATH = 'data/train.txt'
TEST_PATH = 'data/test.txt'

train_raw = load_triplets(TRAIN_PATH)
test_raw = load_triplets(TEST_PATH)

print(f"train: {len(train_raw)} triplets")
print(f"test:  {len(test_raw)} triplets")

train_rels = Counter(r for _, r, _ in train_raw)
test_rels = Counter(r for _, r, _ in test_raw)

print(f"\ntrain relations ({len(train_rels)} types):")
for r, c in train_rels.most_common():
    print(f"  {r}: {c}")

print(f"\ntest relations ({len(test_rels)} types):")
for r, c in test_rels.most_common():
    print(f"  {r}: {c}")

train: 13821 triplets
test:  590 triplets

train relations (28 types):
  grandsonOf: 814
  grandmotherOf: 813
  grandfatherOf: 813
  granddaughterOf: 812
  motherOf: 733
  fatherOf: 733
  sisterOf: 636
  daughterOf: 628
  greatGrandsonOf: 624
  greatGrandmotherOf: 617
  greatGrandfatherOf: 617
  greatGranddaughterOf: 610
  sonOf: 600
  brotherOf: 570
  auntOf: 556
  nephewOf: 514
  nieceOf: 496
  uncleOf: 454
  girlCousinOf: 445
  boyCousinOf: 391
  greatAuntOf: 312
  greatUncleOf: 237
  boyFirstCousinOnceRemovedOf: 180
  secondAuntOf: 175
  secondUncleOf: 158
  girlFirstCousinOnceRemovedOf: 153
  boySecondCousinOf: 68
  girlSecondCousinOf: 62

test relations (4 types):
  sonOf: 214
  daughterOf: 200
  motherOf: 88
  fatherOf: 88


## first observation

the test set only has 4 relation types: motherOf, fatherOf, sonOf, daughterOf.
all parent-child relations. the test is asking if we can recover the core family backbone?

In [3]:
# build ID mappings
entity2id, relation2id, id2entity, id2relation = build_mappings(train_raw, test_raw)
train_ids = triplets_to_ids(train_raw, entity2id, relation2id)
test_ids = triplets_to_ids(test_raw, entity2id, relation2id)

num_entities = len(entity2id)
num_relations = len(relation2id)

print(f"entities:  {num_entities}")
print(f"relations: {num_relations}")

# all known triples for filtering
all_true = set()
for row in train_ids:
    all_true.add(tuple(row))
for row in test_ids:
    all_true.add(tuple(row))

print(f"total known triplets: {len(all_true)}")

entities:  1316
relations: 28
total known triplets: 14411


## baseline 1: random

if we rank entities randomly, what's the expected MRR?
with N entities, expected rank ≈ N/2, so MRR ≈ 2/N.
this is our absolute floor. anything smart should do better than this

In [5]:
# analytical random baseline
expected_rank = num_entities / 2
random_mrr = 1.0 / expected_rank

print(f"num entities: {num_entities}")
print(f"expected rank: {expected_rank:.0f}")
print(f"random MRR: {random_mrr:.6f}")
print(f"random Hits@10: {10/num_entities:.6f}")

print(f"\nso any model with MRR above {random_mrr:.4f} is learning something")

num entities: 1316
expected rank: 658
random MRR: 0.001520
random Hits@10: 0.007599

so any model with MRR above 0.0015 is learning something


## baseline 2: frequency

for each (head, relation), rank candidate tails by how often they appear
as tails of that relation type in the training set.
this captures how some people are more "central" and appear in more relations.

In [6]:
rel_tail_freq = defaultdict(Counter)
for h, r, t in train_ids:
    rel_tail_freq[r][t] += 1

rel_head_freq = defaultdict(Counter)
for h, r, t in train_ids:
    rel_head_freq[r][h] += 1

# also count general tail frequency
tail_freq = Counter()
for h, r, t in train_ids:
    tail_freq[t] += 1


def frequency_rank(h, r, t, all_true_set, num_entities):

    freq = rel_tail_freq[r]
    target_score = freq.get(t, 0)
    
    rank = 1
    for e in range(num_entities):
        if e == t:
            continue
        if (h, r, e) in all_true_set:
            continue
        if freq.get(e, 0) > target_score:
            rank += 1
    return rank


freq_ranks = []
for h, r, t in test_ids:

    rank_t = frequency_rank(h, r, t, all_true, num_entities)
    freq_ranks.append(rank_t)
    
    head_freq = defaultdict(Counter)
    


def frequency_rank_head(h, r, t, all_true_set, num_entities):

    freq = rel_head_freq[r]
    target_score = freq.get(h, 0)
    rank = 1

    for e in range(num_entities):
        if e == h:
            continue
        if (e, r, t) in all_true_set:
            continue
        if freq.get(e, 0) > target_score:
            rank += 1
    return rank

freq_ranks_all = []

for i, (h, r, t) in enumerate(test_ids):

    rank_t = frequency_rank(h, r, t, all_true, num_entities)
    rank_h = frequency_rank_head(h, r, t, all_true, num_entities)
    freq_ranks_all.extend([rank_t, rank_h])
    
    if i % 100 == 0:
        print(f"  {i}/{len(test_ids)}")

freq_metrics = compute_metrics(np.array(freq_ranks_all, dtype=np.float64))
print(f"\nFrequency baseline:")
for k, v in freq_metrics.items():
    print(f"  {k}: {v:.4f}")

  0/590
  100/590
  200/590
  300/590
  400/590
  500/590

Frequency baseline:
  mrr: 0.0058
  hits@1: 0.0000
  hits@3: 0.0000
  hits@10: 0.0119
  mean_rank: 352.2780


## baseline 3: rule-based (connecting to task 3)

we know from task 3 that family relations follow strict logical rules.
the test set only has parent-child relations, so let's use inverse rules:

- if motherOf(A, B) is in train , predict daughterOf(B, A) or sonOf(B, A)
- if fatherOf(A, B) is in train , predict daughterOf(B, A) or sonOf(B, A)
- if sonOf(A, B) is in train , predict motherOf(B, A) or fatherOf(B, A)
- if daughterOf(A, B) is in train , predict motherOf(B, A) or fatherOf(B, A)


In [ ]:
train_lookup = defaultdict(set)
for h, r, t in train_raw:
    train_lookup[(h, r)].add(t)


INVERSE_MAP = {
    'motherOf': ['sonOf', 'daughterOf'],
    'fatherOf': ['sonOf', 'daughterOf'],
    'sonOf': ['motherOf', 'fatherOf'],
    'daughterOf': ['motherOf', 'fatherOf'],
}

rule_hits = 0
rule_total = 0

for h, r, t in test_raw:
    rule_total += 1
    
    #  does any inverse of (h, rel, t) exist in training?
    # inverse means (t, inverse_rel, h) in train 
    found = False
    if r in INVERSE_MAP:
        for inv_r in INVERSE_MAP[r]:
            if h in train_lookup.get((t, inv_r), set()):
                found = True
                break
    
    if found:
        rule_hits += 1

print(f"test triplets recoverable by inverse rules: {rule_hits}/{rule_total}")
print(f"coverage: {rule_hits/rule_total:.2%}")

test triplets recoverable by inverse rules: 590/590
coverage: 100.00%
